In [2]:
import pandas as pd
import requests
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("credentials.json", scope)
client = gspread.authorize(creds)

spreadsheet_url = "https://docs.google.com/spreadsheets/d/1g7uVhRn1czV_kSbaXmjwr-88sxjahTDigl9htQ0xoBw/edit#gid=0"
temp_url = "https://docs.google.com/spreadsheets/d/1vnxBib84T24Hq0BfzWJQobDFaNnmb2A6s36TbLEqrFY/edit?gid=0#gid=0"
holiday_url = "https://docs.google.com/spreadsheets/d/1EZvUyp8vKcgSDke7iclm9b54Sw2JxCG6VzLvGq93XpU/edit?gid=1209652816#gid=1209652816"

spreadsheet = client.open_by_url(spreadsheet_url)
temp = client.open_by_url(temp_url)
holiday = client.open_by_url(holiday_url)

worksheet = spreadsheet.get_worksheet(0)
tempsheet = temp.get_worksheet(0)
holiday = holiday.get_worksheet(0)

data = worksheet.get_all_records()
tdata = tempsheet.get_all_records()
hdata = holiday.get_all_records()

train_data = pd.DataFrame(data)
temp_df = pd.DataFrame(tdata)
holiday_df = pd.DataFrame(hdata)

train_data['sale_date'] = pd.to_datetime(train_data['sale_date'], errors='coerce')

if train_data['sale_date'].isnull().sum() > 0:
    print("Warning: Some dates could not be converted. Check for incorrect formats.")

train_data['ปี'] = train_data['sale_date'].dt.strftime('%Y')
train_data['เดือน'] = train_data['sale_date'].dt.strftime('%m')
train_data['เดือน-ปี'] = train_data['sale_date'].dt.strftime('%m-%Y')
train_data['เดือน-วัน'] = train_data['sale_date'].dt.strftime('%m-%d')


train_data = pd.merge(train_data, holiday_df, how='left', left_on='เดือน-วัน', right_on='วันที่')
train_data['holiday'] = train_data['วันสำคัญ'].notnull()
train_data['holiday'] = train_data['holiday'].replace({True: 1, False: 0})
train_data = pd.merge(train_data, temp_df, how='left', left_on='เดือน-ปี', right_on='เดือนอุณหภูมิ')

rainfall_data = {}
for year in range(2018, 2025):
    url = f"https://data.tmd.go.th/api/ThailandMonthlyRainfall/v1/index.php?uid=api&ukey=api12345&format=json&year={year}"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        yearly_rainfall = {}
        for station in data.get('StationMonthlyRainfall', []):
            province = station['StationNameThai']
            rainfall = station['MonthlyRainfall']
            yearly_rainfall[province] = {str(i+1).zfill(2): float(rainfall[f'Rainfall{month}']) for i, month in enumerate(
                ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
            )}
        rainfall_data[year] = yearly_rainfall
    else:
        print(f"Failed to fetch rainfall data for {year}")

province = "นครราชสีมา"

train_data['ปริมาณน้ำฝน'] = train_data.apply(lambda row: rainfall_data.get(int(row['ปี']), {}).get(province, {}).get(row['เดือน'], 0.0), axis=1)
train_data.drop(columns=['เดือน', 'ปี'], inplace=True)
train_data = train_data.drop(columns=['เดือนอุณหภูมิ'])
train_data = train_data.drop(columns=['เดือน-ปี'])
train_data = train_data.drop(columns=['เดือน-วัน'])
train_data = train_data.drop(columns=['วันที่'])
train_data.to_csv("cleaned_train_data.csv", index=False)

def updateToGSheetFromCSV(csv_file_path):
    datacleaned_url = "https://docs.google.com/spreadsheets/d/1eohhRmXeB-e7xpoacjfbRIA2Igz8GP7A1Sbe3ekqAdQ/edit?gid=0#gid=0"
    datacleaned_spreadsheet = client.open_by_url(datacleaned_url)
    datacleaned_worksheet = datacleaned_spreadsheet.get_worksheet(0)
    
    data_from_csv = pd.read_csv(csv_file_path)
    data_from_csv = data_from_csv.fillna('')
    datacleaned_worksheet.clear()
    datacleaned_worksheet.update('A1', [data_from_csv.columns.tolist()] + data_from_csv.values.tolist())
    
    print("Data successfully uploaded to Google Sheets.")

# train_data.to_csv("cleaned_train_data.csv", index=False)

# Now upload the CSV to Google Sheets
updateToGSheetFromCSV("cleaned_train_data.csv")
train_data

C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\3212883669.py:43: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['holiday'] = train_data['holiday'].replace({True: 1, False: 0})
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\3212883669.py:81: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  datacleaned_worksheet.update('A1', [data_from_csv.columns.tolist()] + data_from_csv.values.tolist())


Data successfully uploaded to Google Sheets.


,invoice_no,sale_date,sale_time,customer_id,drug_code,product_name,batch_no,storage_location,quantity,unit,unit_price,total_amount,gross_profit,วันสำคัญ,holiday,อุณหภูมิ,ปริมาณน้ำฝน
0,BL180101-001,2018-01-01,8:44,C01-0000,S0125,SULIDINE SYRUP 12X60CC,P171227-0002,B1,1,ขวด,35.0,35,14.17,วันขึ้นปีใหม่,1,25.6,2.6
1,BL180101-005,2018-01-01,9:45,C01-0000,C0142,CETRIzin แผง,P171226-0001,B3,1,แผง,20.0,20,13.50,วันขึ้นปีใหม่,1,25.6,2.6
2,BL180101-007,2018-01-01,10:00,C01-0000,C0142,CETRIzin แผง,P171226-0001,B3,1,แผง,20.0,20,13.50,วันขึ้นปีใหม่,1,25.6,2.6
3,BL180101-013,2018-01-01,11:20,C01-0000,G0074,Gaviscon ซอง,P171024-0015,F2,2,ซอง,20.0,40,9.17,วันขึ้นปีใหม่,1,25.6,2.6
4,BL180101-015,2018-01-01,11:56,C01-0000,B0086,Biocalm ขายแผง,P151126-0005,B4,1,แผง,30.0,30,20.80,วันขึ้นปีใหม่,1,25.6,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12800,BL201231-046,2020-12-31,18:22,C01-0000,C0142,CETRIzin แผง,P201215-0061,F2,1,แผง,20.0,20,13.60,วันสิ้นปี,1,24.4,0.0
12801,BL201231-048,2020-12-31,18:34,C01-0000,C0142,CETRIzin แผง,P201215-0061,F2,1,แผง,20.0,20,13.60,วันสิ้นปี,1,24.4,0.0
12802,BL201231-049,2020-12-31,18:54,C01-0000,I0108,Ibrofen 600 TO แผง,P201215-0028,F2,1,แผง,40.0,40,26.00,วันสิ้นปี,1,24.4,0.0
12803,BL201231-054,2020-12-31,19:53,C01-0000,A0158,ยาคุม ANNA 28 เม็ด,P201110-0017,F2,2,แผง,35.0,70,21.60,วันสิ้นปี,1,24.4,0.0


In [8]:
import pandas as pd

df = train_data
df['sale_date'] = pd.to_datetime(df['sale_date'], format='%Y-%m-%d')

df_grouped = df.groupby(['sale_date', 'drug_code'], as_index=False)['quantity'].sum()
df_grouped = df_grouped.merge(df[['sale_date', 'holiday', 'อุณหภูมิ', 'ปริมาณน้ำฝน']].drop_duplicates(), on='sale_date', how='left')
df_grouped = df_grouped.sort_values(by='sale_date')

df_grouped.to_csv('processed_sales_data.csv', index=False)
df_grouped

,sale_date,drug_code,quantity,holiday,อุณหภูมิ,ปริมาณน้ำฝน
0,2018-01-01,A0158,1,1,25.6,2.6
1,2018-01-01,B0086,1,1,25.6,2.6
2,2018-01-01,C0142,9,1,25.6,2.6
3,2018-01-01,C0173,1,1,25.6,2.6
4,2018-01-01,D0160,6,1,25.6,2.6
...,...,...,...,...,...,...
8430,2020-12-31,C0142,9,1,24.4,0.0
8431,2020-12-31,C0173,1,1,24.4,0.0
8432,2020-12-31,D0189,1,1,24.4,0.0
8433,2020-12-31,E0055,1,1,24.4,0.0


In [10]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input
import gspread
from oauth2client.service_account import ServiceAccountCredentials

name = "A0175"

df = df_grouped  
df['sale_date'] = pd.to_datetime(df['sale_date'])
df = df.sort_values(by='sale_date')
    
df_drug = df[df['drug_code'] == name].copy()
df_drug['year_month'] = df_drug['sale_date'].dt.to_period('M')

df_monthly = df_drug.groupby('year_month').agg({
    'quantity': 'sum',
    'อุณหภูมิ': 'mean',
    'ปริมาณน้ำฝน': 'mean'
}).reset_index()

df_monthly['year_month'] = df_monthly['year_month'].astype(str)

scaler_quantity = MinMaxScaler()
df_monthly['quantity'] = scaler_quantity.fit_transform(df_monthly[['quantity']])

scaler_features = MinMaxScaler()
df_monthly[['อุณหภูมิ', 'ปริมาณน้ำฝน']] = scaler_features.fit_transform(df_monthly[['อุณหภูมิ', 'ปริมาณน้ำฝน']])

seq_length = 24

def create_sequence(data, seq_length=seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length][['quantity', 'อุณหภูมิ', 'ปริมาณน้ำฝน']].values)
        y.append(data.iloc[i+seq_length]['quantity'])
    return np.array(X), np.array(y)

X, y = create_sequence(df_monthly, seq_length)

if X.shape[0] == 0:
    raise ValueError("Insufficient data for sequence creation. Adjust seq_length.")

model = Sequential([
    Input(shape=(X.shape[1], X.shape[2])),
    LSTM(units=100, return_sequences=True),
    LSTM(units=50, return_sequences=False),
    Dense(units=1)
])

model.compile(optimizer='adam', loss='mean_squared_error')

model.fit(X, y, epochs=300, batch_size=16, verbose=1)

num_months = 24  
last_months = df_monthly.tail(seq_length)[['quantity', 'อุณหภูมิ', 'ปริมาณน้ำฝน']].values

predictions = []
for i in range(num_months):
    last_months_reshaped = np.reshape(last_months, (1, last_months.shape[0], last_months.shape[1]))
    predicted_quantity = model.predict(last_months_reshaped)[0][0]

    if i < 12:
        temp_rain = df_monthly.iloc[-12 + i][['อุณหภูมิ', 'ปริมาณน้ำฝน']].values
    else:
        temp_rain = df_monthly[['อุณหภูมิ', 'ปริมาณน้ำฝน']].mean().values  

    predictions.append(abs(predicted_quantity))
    last_months = np.vstack([last_months[1:], [predicted_quantity, temp_rain[0], temp_rain[1]]])

# predictions = scaler_quantity.inverse_transform(np.array(predictions).reshape(-1, 1)).flatten()

dates = pd.date_range(start='2020-01', periods=num_months, freq='M')
predictions_df = pd.DataFrame({'date': dates, 'predicted_quantity': predictions})
predictions_df['date'] = predictions_df['date'].dt.strftime('%Y-%m')

predictions_df.to_csv(f'predictions_{name}_2020_2021_monthly.csv', index=False)

scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name('credentials.json', scope)
client = gspread.authorize(creds)

spreadsheet_id = "13_f_4pEUgCifgGYRzME0eFNRVrt_vGMKEcygE-MrFJ8"
spreadsheet = client.open_by_key(spreadsheet_id)

try:
    sheet = spreadsheet.worksheet(name)
    spreadsheet.del_worksheet(sheet)  
except gspread.WorksheetNotFound:
    pass  

sheet = spreadsheet.add_worksheet(title=name, rows=str(len(predictions_df) + 1), cols="2")
sheet.update('A1', [['date', 'predicted_quantity']])
sheet.update('A2', predictions_df.values.tolist())

print(f"📊 Monthly predictions for {name} (2020-2021) saved to Google Sheets successfully! 🚀")

Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.1543
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0659
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - loss: 0.0221
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.0241
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - loss: 0.0441
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step - loss: 0.0439
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - loss: 0.0323
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - loss: 0.0221
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0175
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step - loss: 0.0177
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0202
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0228
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0244
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0247
Epoch 15/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0238
Epoch 16/300
1/1 ━━━━

C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:81: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2020-01', periods=num_months, freq='M')
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:101: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [['date', 'predicted_quantity']])
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:102: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A2', predictions_df.values.tolist())


📊 Monthly predictions for A0158 (2020-2021) saved to Google Sheets successfully! 🚀
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0910
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.0527
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0423
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0520
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0533
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0467
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0406
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0379
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0379
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0388
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0394
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0390
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0376
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0356
Epoc

C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:81: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2020-01', periods=num_months, freq='M')
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:101: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [['date', 'predicted_quantity']])
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:102: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A2', predictions_df.values.tolist())


📊 Monthly predictions for A0160 (2020-2021) saved to Google Sheets successfully! 🚀
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - loss: 0.3640
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step - loss: 0.2191
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.1163
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0548
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0421
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0726
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0887
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - loss: 0.0782
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0599
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0466
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0410
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0414
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step - loss: 0.0446
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - loss: 0.0485
Epoc

...

📊 Monthly predictions for I0108 (2020-2021) saved to Google Sheets successfully! 🚀
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.1236
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0642
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0349
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0357
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0488
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0488
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.0415
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0350
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0320
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0318
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 0.0331
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0347
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0358
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0361
Epoc

C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:81: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2020-01', periods=num_months, freq='M')
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:101: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [['date', 'predicted_quantity']])
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:102: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A2', predictions_df.values.tolist())


📊 Monthly predictions for O0045 (2020-2021) saved to Google Sheets successfully! 🚀
Epoch 1/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - loss: 0.0623
Epoch 2/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0336
Epoch 3/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0232
Epoch 4/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - loss: 0.0276
Epoch 5/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - loss: 0.0313
Epoch 6/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 0.0288
Epoch 7/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - loss: 0.0248
Epoch 8/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - loss: 0.0221
Epoch 9/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - loss: 0.0210
Epoch 10/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - loss: 0.0211
Epoch 11/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0216
Epoch 12/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0220
Epoch 13/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - loss: 0.0220
Epoch 14/300
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - loss: 0.0216
Epoc

C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:81: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(start='2020-01', periods=num_months, freq='M')
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:101: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A1', [['date', 'predicted_quantity']])
C:\Users\Admin\AppData\Local\Temp\ipykernel_27092\1119413837.py:102: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  sheet.update('A2', predictions_df.values.tolist())


📊 Monthly predictions for S0125 (2020-2021) saved to Google Sheets successfully! 🚀


In [31]:
print(f"Length of dates_2021: {len(dates_2021)}")  # Should be 730
print(f"Length of predictions_2021: {len(predictions_2021)}")  # Should also be 730


Length of dates_2021: 731
Length of predictions_2021: 730


In [18]:
target_drug_code = "A0178"

df_target_drug = df[df['drug_code'] == target_drug_code]
row_count = len(df_target_drug)
print(row_count)


82


In [58]:
df_monthly

,year_month,quantity,อุณหภูมิ,ปริมาณน้ำฝน
0,2018-01,0.282787,0.000000,0.000000
1,2018-02,0.000000,0.000000,0.518615
2,2018-03,0.344262,0.486486,0.176925
3,2018-04,0.159836,0.837838,0.945010
4,2018-05,0.495902,0.189189,0.816313
5,2018-06,0.385246,1.000000,0.636656
6,2018-07,0.225410,0.891892,1.000000
7,2018-08,1.000000,0.783784,0.576405
8,2018-09,0.508197,0.702703,0.932577
9,2018-10,0.467213,0.162162,0.711934
